In [1]:
import pygame
from pygame.locals import *

import matplotlib.pyplot as plt

import os

%matplotlib inline

import pandas as pd
import numpy as np

import tensorflow as tf


moves = pd.read_csv('moves.csv')
df = pd.DataFrame(moves, index = [i for i in range(0, 2438)])


df_filteredW = df.query("CLASS == 'win'",inplace=False)
df_filteredL = df.query("CLASS == 'loss'",inplace= False)
df_filteredD = df.query("CLASS == 'draw'",inplace = False)

win_list = []
for row in df_filteredW.values:
    print ("a",row) 
    temp_list = []
    for item in row:
        if item != '?' and item != 'win' and item != 'draw' and item != 'loss':
            temp_list.append(int(item))
    win_list.append(temp_list)
 
            
draw_list = []
for row in df_filteredD.values:
    print ("a",row) 
    temp2_list = []
    for item in row:
        if item != '?' and item != 'win' and item != 'draw' and item != 'loss':
            temp2_list.append(int(item))
    draw_list.append(temp2_list)



print(win_list[0])
print(draw_list[0])

print(df_filteredW)
print(df_filteredD)
print(df_filteredL)

running = False

mv = []

for i in range (1, 8):
    mv.append('MOVE' + str(i)) 
    print(mv)
    if mv[i - 1] == "?":
        running = True

    else:
        running = False

current_game = ['?', '?', '?', '?', '?', '?', '?', '?']
current_move = 0

        
        
# Create the constants (go ahead and experiment with dirent values)
BOARDWIDTH = 3  # number of columns in the board
BOARDHEIGHT = 3 # number of rows in the board
TILESIZE = 150
WINDOWWIDTH = 600
WINDOWHEIGHT = 600
FPS = 60
BLANK = None

#                 R    G    B
BLACK =         (  0,   0,   0)
WHITE =         (255, 255, 255)
BLUE =         (  0,   0,   255)
RED =         (255,   0,   0)

BGCOLOR = WHITE
TILECOLOR = WHITE
TEXTCOLOR = BLACK
BORDERCOLOR = BLACK
BASICFONTSIZE = 20
OTHERFONTSIZE = 120

BUTTONCOLOR = WHITE
#BUTTONTEXTCOLOR = BLACK
MESSAGECOLOR = WHITE

BLANK = 10
PLAYER_O = 11
PLAYER_X = 21


PLAYER_O_WIN = PLAYER_O * 3
PLAYER_X_WIN = PLAYER_X * 3

CONT_GAME         = 10
DRAW_GAME         = 20
QUIT_GAME         = 30

XMARGIN = int((WINDOWWIDTH - (TILESIZE * BOARDWIDTH + (BOARDWIDTH - 1))) / 2)
YMARGIN = int((WINDOWHEIGHT - (TILESIZE * BOARDHEIGHT + (BOARDHEIGHT - 1))) / 2)

choice = 0


def winning_move(board):
    def check_draw_game():
        return sum(board)%10 == 9

    def check_horizontal(player):
        for i in [0, 3, 6]:
            if sum(board[i:i+3]) == 3 * player:
                return player

    def check_vertical(player):
        for i in range(3):
            if sum(board[i::3]) == 3 * player:
                return player

    def check_diagonals(player):
        if (sum(board[0::4]) == 3 * player) or (sum(board[2:7:2]) == 3 * player):
            return player

    for player in [PLAYER_X, PLAYER_O]:
        if any([check_horizontal(player), check_vertical(player), check_diagonals(player)]):
            return player

    return DRAW_GAME if check_draw_game() else CONT_GAME


# def unit_score(winner, depth):
#     if winner == DRAW_GAME:
#         return 0
#     else:
#         return 10 - depth if winner == PLAYER_X else depth - 10


def get_available_step(board):
    return [i for i in range(9) if board[i] == BLANK]


# def minmax(board, depth):
#     global choice
#     result = winning_move(board)
#     if result != CONT_GAME:
#         return unit_score(result, depth)

#     depth += 1
#     scores = []
#     steps = []

#     for step in get_available_step(board):
#         score = minmax(update_state(board, step, depth), depth)
#         scores.append(score)
#         steps.append(step)

#     if depth % 2 == 1:
#         max_value_index = scores.index(max(scores))
#         choice = steps[max_value_index]
#         return max(scores)
#     else:
#         min_value_index = scores.index(min(scores))
#         choice = steps[min_value_index]
#         return min(scores)


def update_state(board, step, depth):
    board = list(board)
    board[step] = PLAYER_X if depth % 2 else PLAYER_O
    return board


def update_board(board, step, player):
    global current_game
    global current_move
    
    board[step] = player
    current_game[current_move] = step
    current_move += 1

    
    

def change_to_player(player):
    if player == PLAYER_O:
        return 'O'
    elif player == PLAYER_X:
        return 'X'
    elif player == BLANK:
        return '-'


def drawBoard(board, message):
    DISPLAYSURF.fill(BGCOLOR)
    # game_board = pygame.image.load('./ticgameboard2.png')
    # DISPLAYSURF.blit(game_board, (140, 140))
    if message:
        textSurf, textRect = makeText(message, MESSAGECOLOR, BGCOLOR, 5, 5)
        DISPLAYSURF.blit(textSurf, textRect)

    for tilex in range(3):
        for tiley in range(3):
            if board[tilex*3+tiley] != BLANK:
                drawTile(tilex, tiley, board[tilex*3+tiley])

    left, top = getLeftTopOfTile(0, 0)
    width = BOARDWIDTH * TILESIZE
    height = BOARDHEIGHT * TILESIZE
    # pygame.draw.rect(DISPLAYSURF, BORDERCOLOR, (left - 5, top - 5, width + 11, height + 11), 4)
    pygame.draw.line(DISPLAYSURF, BORDERCOLOR, (225, 525), (225, 75), 5)
    pygame.draw.line(DISPLAYSURF, BORDERCOLOR, (375, 525), (375, 75), 5)
    pygame.draw.line(DISPLAYSURF, BORDERCOLOR, (75, 225), (525, 225), 5)
    pygame.draw.line(DISPLAYSURF, BORDERCOLOR, (75, 375), (525, 375), 5)
    DISPLAYSURF.blit(NEW_SURF, NEW_RECT)
    DISPLAYSURF.blit(NEW_SURF2, NEW_RECT2)
    DISPLAYSURF.blit(NEW_SURF3, NEW_RECT3)


def getLeftTopOfTile(tileX, tileY):
    left = XMARGIN + (tileX * TILESIZE) + (tileX - 1)
    top = YMARGIN + (tileY * TILESIZE) + (tileY - 1)
    return (left, top)


def makeText(text, color, bgcolor, top, left):
    '''Create the Surface and Rect objects for some text.'''
    textSurf = BASICFONT.render(text, True, color, bgcolor)
    textRect = textSurf.get_rect()
    textRect.topleft = (top, left)
    return (textSurf, textRect)


def drawTile(tilex, tiley, symbol, adjx=0, adjy=0):
    '''
    Draw a tile at board coordinates tilex and tiley, optionally a few
    pixels over (determined by adjx and adjy).
    '''
    left, top = getLeftTopOfTile(tilex, tiley)
    pygame.draw.rect(DISPLAYSURF, TILECOLOR, (left + adjx, top + adjy, TILESIZE, TILESIZE))
    if symbol == PLAYER_X:
        textSurf = BASICFONT.render(symbol_to_str(symbol), True, BLUE)
    elif symbol == PLAYER_O:
        textSurf = BASICFONT.render(symbol_to_str(symbol), True, RED)
    textRect = textSurf.get_rect()
    textRect.center = left + int(TILESIZE / 2) + adjx, top + int(TILESIZE / 2) + adjy
    DISPLAYSURF.blit(textSurf, textRect)


def symbol_to_str(symbol):
    if symbol == PLAYER_O:
        return 'O'
    elif symbol == PLAYER_X:
        return 'X'


def getSpotClicked(x, y):
    '''From the x & y pixel coordinates, get the x & y board coordinates.'''
    for tileX in range(3):
        for tileY in range(3):
            left, top = getLeftTopOfTile(tileX, tileY)
            tileRect = pygame.Rect(left, top, TILESIZE, TILESIZE)
            if tileRect.collidepoint(x, y):
                return (tileX, tileY)
    return None

def check_next_spot(current_game, current_move, win_list, draw_list):
    #check if board is in win position
    for item in win_list:
        i = 0
        while i < current_move:
            if item[i] != current_game[i]:
                break
            i += 1
        current_game[current_move] = item[current_move]
        is_move = True
        break
   
    if is_move == True:
        return item[current_move]
    else:
        for item in draw_list:
            i = 0
            while i < current_move:
                if item[i] != current_game[i]:
                    break
                i += 1
            current_game[current_move] = item[current_move]
            is_move = True
            return item[current_move]
            
def board_to_step(spotx, spoty):
    return spotx * 3 + spoty


def check_move_legal(coords, board):
    step = board_to_step(*coords)
    return board[step] == BLANK

def main():
    global FPSCLOCK, DISPLAYSURF, BASICFONT, NEW_SURF, NEW_RECT, NEW_SURF2, NEW_RECT2, NEW_SURF3, NEW_RECT3
    two_player = False #by default false
    pygame.init()
    FPSCLOCK = pygame.time.Clock()
    DISPLAYSURF = pygame.display.set_mode((WINDOWWIDTH, WINDOWHEIGHT))
    pygame.display.set_caption('Unbeatable')
    BASICFONT = pygame.font.Font('freesansbold.ttf', BASICFONTSIZE)
    OTHERFONT = pygame.font.Font('freesansbold.ttf', OTHERFONTSIZE)
    NEW_SURF, NEW_RECT = makeText('Unbeatable', RED, TILECOLOR, 400, 540)
    NEW_SURF2, NEW_RECT2 = makeText('Easy', BLUE, TILECOLOR, 120, 540)
    NEW_SURF3, NEW_RECT3 = makeText('Tic-Tac-Toe AI Game', TEXTCOLOR, TILECOLOR, 200, 20)
    board = [BLANK] * 9
    game_over = False
    x_turn = True
    msg = "AI Tic-Tac-Toe"
    drawBoard(board, msg)
    pygame.display.update()

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                running = False
                
        coords = None
        for event in pygame.event.get():
            if event.type == MOUSEBUTTONUP:
                coords = getSpotClicked(event.pos[0], event.pos[1])
                if not coords and NEW_RECT.collidepoint(event.pos):
                    board = [BLANK] * 9
                    game_over = False
                    msg = "Unbeatable Tic Tac Toe AI"
                    drawBoard(board, msg)
                    pygame.display.update()
                    two_player = False
                if not coords and NEW_RECT2.collidepoint(event.pos):
                    board = [BLANK] * 9
                    game_over = False
                    msg = "Unbeatable Tic Tac Toe AI"
                    drawBoard(board, msg)
                    pygame.display.update()
                    two_player = True
        if coords and check_move_legal(coords, board) and not game_over:
            if two_player:
                next_step = board_to_step(*coords)
                if x_turn:
                    update_board(board, next_step, PLAYER_X)
                    x_turn = False
                else:
                    update_board(board, next_step, PLAYER_O)
                    x_turn = True
                drawBoard(board, msg)
                pygame.display.update()

            if not two_player:
                while not game_over:
                    next_step = check_next_spot(current_game, current_move, win_list, draw_list)
#                 next_step = board_to_step(*coords)
                    update_board(board, next_step, PLAYER_X)
                    drawBoard(board, msg)
                    pygame.display.update()
#                 minmax(board, 0)
                    update_board(board, choice, PLAYER_O)

            result = winning_move(board)
            game_over = (result != CONT_GAME)

            if result == PLAYER_X:
                msg = "X wins!"
            elif result == PLAYER_O:
                msg = "O wins!"
            elif result == DRAW_GAME:
                msg = "Draw game"

            drawBoard(board, msg)
            pygame.display.update()


if __name__ == '__main__':
    main()

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
a [4.0 '7' '2' '6' '?' '?' '?' 'win']
a [0.0 '4' '2' '1' '?' '?' '?' 'win']
a [4.0 '0' '8' '?' '?' '?' '?' 'win']
a [4.0 '6' '1' '7' '?' '?' '?' 'win']
a [6.0 '8' '7' '?' '?' '?' '?' 'win']
a [6.0 '4' '1' '5' '2' '?' '?' 'win']
a [7.0 '2' '5' '8' '?' '?' '?' 'win']
a [1.0 '8' '?' '?' '?' '?' '?' 'win']
a [2.0 '5' '3' '?' '?' '?' '?' 'win']
a [4.0 '3' '5' '?' '?' '?' '?' 'win']
a [2.0 '8' '5' '?' '?' '?' '?' 'win']
a [5.0 '8' '7' '?' '?' '?' '?' 'win']
a [4.0 '2' '0' '5' '?' '?' '?' 'win']
a [8.0 '6' '5' '?' '?' '?' '?' 'win']
a [3.0 '1' '?' '?' '?' '?' '?' 'win']
a [1.0 '6' '7' '?' '?' '?' '?' 'win']
a [0.0 '4' '7' '?' '?' '?' '?' 'win']
a [4.0 '8' '2' '7' '?' '?' '?' 'win']
a [5.0 '7' '2' '?' '?' '?' '?' 'win']
a [6.0 '8' '3' '7' '?' '?' '?' 'win']
a [1.0 '5' '4' '7' '?' '?' '?' 'win']
a [2.0 '8' '7' '?' '?' '?' '?' 'win']
a [1.0 '8' '7' '?' '?' '?' '?' 'win']
a [2.0 '8' '7' '?' '?' '?' '?' 'win']
a [

a [2.0 '8' '3' '7' '?' '?' '?' 'draw']
a [4.0 '7' '0' '5' '6' '?' '?' 'draw']
a [2.0 '1' '8' '3' '?' '?' '?' 'draw']
a [4.0 '1' '6' '5' '0' '?' '?' 'draw']
a [8.0 '7' '2' '3' '?' '?' '?' 'draw']
a [4.0 '3' '2' '7' '0' '?' '?' 'draw']
a [8.0 '5' '6' '1' '?' '?' '?' 'draw']
a [4.0 '8' '3' '2' '7' '?' '?' 'draw']
a [6.0 '0' '5' '1' '?' '?' '?' 'draw']
a [4.0 '2' '7' '3' '0' '?' '?' 'draw']
a [8.0 '6' '1' '5' '?' '?' '?' 'draw']
a [1.0 '2' '8' '3' '7' '?' '?' 'draw']
a [4.0 '0' '5' '6' '?' '?' '?' 'draw']
a [2.0 '5' '0' '7' '3' '?' '?' 'draw']
a [4.0 '8' '1' '6' '?' '?' '?' 'draw']
a [5.0 '1' '6' '0' '7' '?' '?' 'draw']
a [4.0 '2' '8' '3' '?' '?' '?' 'draw']
a [4.0 '2' '3' '8' '1' '?' '?' 'draw']
a [0.0 '6' '5' '7' '?' '?' '?' 'draw']
a [4.0 '1' '8' '6' '3' '?' '?' 'draw']
a [2.0 '7' '0' '5' '?' '?' '?' 'draw']
a [4.0 '7' '2' '3' '8' '?' '?' 'draw']
a [0.0 '1' '6' '5' '?' '?' '?' 'draw']
a [8.0 '1' '6' '5' '3' '?' '?' 'draw']
a [4.0 '2' '7' '0' '?' '?' '?' 'draw']
a [4.0 '2' '7' '3' '0' '?

IndexError: list index out of range